# Tutorial - Ensemble - Classification


We will predict the price category, among 4 categories, of an AIRBNB listing (`price_category` column). This is a multi-class classification task.

**The unit of analysis is an AIRBNB LISTING**

# Setup

In [ ]:
# Common imports
import numpy as np
import pandas as pd

np.random.seed(42)


# Get the data

In [ ]:
#We will predict the "price_gte_150" value in the data set:

airbnb = pd.read_csv("airbnb.csv")
airbnb.head()

# Split the data into train and test

In [ ]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(airbnb, test_size=0.3)

### Be careful: we haven't seperated the target column yet

## Check the missing values

In [ ]:
train_set.isna().sum()

In [ ]:
test_set.isna().sum()

# Data Prep

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import FunctionTransformer

## Drop the variables we can't use in this tutorial

In [ ]:
# We can't use the following columns in this tutorial, because they are not for binary classification tasks

train = train_set.drop(['price', 'price_gte_150'], axis=1)
test = test_set.drop(['price', 'price_gte_150'], axis=1)

## Separate the target variable (we don't want to transform it)

In [ ]:
train_y = train['price_category']
test_y = test['price_category']

train_inputs = train.drop(['price_category'], axis=1)
test_inputs = test.drop(['price_category'], axis=1)

## Feature Engineering: Let's derive a new column

### Inspect the `number of reviews` column

In [ ]:
train_inputs['number_of_reviews'].describe()

In [ ]:
train_inputs['number_of_reviews'].hist(bins=50)

### Create a new ordinal column: convert the number of reviews to ordinal values.

In [ ]:
def new_col(df):
    #Create a copy so that we don't overwrite the existing dataframe
    df1 = df.copy()
    
    df1['num_reviews_binned'] = pd.cut(df1['number_of_reviews'],
                                       bins=[0,0.5,1,5,15,50,10000],  #bins=[exclusive, inclusive]
                                       labels=False, 
                                       include_lowest=True,
                                       ordered=True)
    
#     You can also do this if you want categorical values:    
#     df1['num_reviews_binned'] = pd.cut(df1['number_of_reviews'],
#                                        bins=[0,0.5,1,5,15,50,10000], 
#                                        labels=['None','Very few','Few','Medium','Many','Too many'], 
#                                        include_lowest=True,
#                                        ordered=False)

    
    return df1[['num_reviews_binned']]
    # You can use this to check whether the calculation is made correctly:
    #return df1

In [ ]:
#Let's test the new function:

# Send train set to the function we created
new_col(train_set)

In [ ]:
#Check the new distribution

new_col(train_set).value_counts()

##  Identify the numerical and categorical columns

In [ ]:
train_inputs.dtypes

**At this stage, you can manually identify numeric, binary, and categorical columns as follows:**

`numeric_columns = ['latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'Number of amenities', 'guests_included', 'price_per_extra_person', 'minimum_nights', 'number_of_reviews', 'number_days_btw_first_last_review', 'review_scores_rating']`
 
 `binary_columns = ['host_is_superhost', 'host_identity_verified']`
 
 `categorical_columns = ['neighbourhood_cleansed', 'property_type', 'room_type', 'bed_type', 'cancellation_policy']`
 
<br>
 
**If you do not want to manually type these, you can do the below tricks:**

In [ ]:
# Identify the numerical columns
numeric_columns = train_inputs.select_dtypes(include=[np.number]).columns.to_list()

# Identify the categorical columns
categorical_columns = train_inputs.select_dtypes('object').columns.to_list()

In [ ]:
# Identify the binary columns so we can pass them through without transforming
binary_columns = ['host_is_superhost', 'host_identity_verified']

In [ ]:
# Be careful: numerical columns already includes the binary columns,
# So, we need to remove the binary columns from numerical columns.

for col in binary_columns:
    numeric_columns.remove(col)

In [ ]:
binary_columns

In [ ]:
numeric_columns

In [ ]:
categorical_columns

In [ ]:
feat_eng_columns = ['number_of_reviews']

# Pipeline

In [ ]:
numeric_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [ ]:
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='unknown')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

In [ ]:
binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))])

In [ ]:
my_new_column = Pipeline(steps=[('my_new_column', FunctionTransformer(new_col)),
                               ('scaler', StandardScaler())])

In [ ]:
preprocessor = ColumnTransformer([
        ('num', numeric_transformer, numeric_columns),
        ('cat', categorical_transformer, categorical_columns),
        ('binary', binary_transformer, binary_columns),
        ('trans', my_new_column, feat_eng_columns)],
        remainder='passthrough')

#passtrough is an optional step. You don't have to use it.

# Transform: fit_transform() for TRAIN

In [ ]:
#Fit and transform the train data
train_x = preprocessor.fit_transform(train_inputs)

train_x

In [ ]:
train_x.shape

# Tranform: transform() for TEST

In [ ]:
# Transform the test data
test_x = preprocessor.transform(test_inputs)

test_x

In [ ]:
test_x.shape

# Baseline

In [ ]:
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")

dummy_clf.fit(train_x, train_y)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
#Baseline Train Accuracy
dummy_train_pred = dummy_clf.predict(train_x)

baseline_train_acc = accuracy_score(train_y, dummy_train_pred)

print('Baseline Train Accuracy: {}' .format(baseline_train_acc))

In [ ]:
#Baseline Test Accuracy
dummy_test_pred = dummy_clf.predict(test_x)

baseline_test_acc = accuracy_score(test_y, dummy_test_pred)

print('Baseline Test Accuracy: {}' .format(baseline_test_acc))

# Multiclass classification



## Hard voting

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import SGDClassifier 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier


dtree_clf = DecisionTreeClassifier(max_depth=20)
log_clf = LogisticRegression(multi_class='multinomial', solver = 'lbfgs', C=10, max_iter=1000)
sgd_clf = SGDClassifier(max_iter=10000, tol=1e-3)

voting_clf = VotingClassifier(
            estimators=[('dt', dtree_clf), 
                        ('lr', log_clf), 
                        ('sgd', sgd_clf)],
            voting='hard')

voting_clf.fit(train_x, train_y)

## Accuracy

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
#Train accuracy

train_y_pred = voting_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = voting_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

## Classification Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(test_y, test_y_pred)

## Inspect each classifier's accuracy

In [ ]:
for clf in (dtree_clf, log_clf, sgd_clf, voting_clf):
    clf.fit(train_x, train_y.ravel())
    test_y_pred = clf.predict(test_x)
    print(clf.__class__.__name__, 'Test acc=', accuracy_score(test_y, test_y_pred))

## Hard voting with weights

In [ ]:
dtree_clf = DecisionTreeClassifier(max_depth=20)
log_clf = LogisticRegression(multi_class='multinomial', solver = 'lbfgs', C=10, max_iter=1000)
sgd_clf = SGDClassifier(max_iter=10000, tol=1e-3)

voting_clf = VotingClassifier(
            estimators=[('dt', dtree_clf), 
                        ('lr', log_clf), 
                        ('sgd', sgd_clf)],
            voting='hard',
            weights=[0.6, 0.2, 0.2])

voting_clf.fit(train_x, train_y)

In [ ]:
#Train accuracy

train_y_pred = voting_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = voting_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

## Soft voting

In [ ]:
#Each model should have predict_proba() function. Otherwise, you can't use it for soft voting
#We can't use sgd, because it doesn't have predict_proba() function.

voting_clf = VotingClassifier(
            estimators=[('dt', dtree_clf), 
                        ('lr', log_clf)],
            voting='soft')

voting_clf.fit(train_x, train_y)

In [ ]:
#Train accuracy

train_y_pred = voting_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = voting_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

# Bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier 


#If you want to do pasting, change "bootstrap=False"
#n_jobs=-1 means use all CPU cores
#bagging automatically performs soft voting

bag_clf = BaggingClassifier( 
            SGDClassifier(), n_estimators=50, 
            max_samples=1000, bootstrap=True, n_jobs=-1) 

bag_clf.fit(train_x, train_y)



In [ ]:
#Train accuracy

train_y_pred = bag_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = bag_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

## Random Patches

In [ ]:
#Random Patches: see the max_features variable
bag_clf = BaggingClassifier( 
            SGDClassifier(), n_estimators=50, max_features=10,
            max_samples=1000, bootstrap=True, n_jobs=-1) 

bag_clf.fit(train_x, train_y)

In [ ]:
#Train accuracy

train_y_pred = bag_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = bag_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

## Random Subspaces

In [ ]:
#Random subspaces: see the max_samples variable
bag_clf = BaggingClassifier( 
            SGDClassifier(), n_estimators=50, max_features=10,
            max_samples=len(train_x), bootstrap=True, n_jobs=-1) 

bag_clf.fit(train_x, train_y)

In [ ]:
#Train accuracy

train_y_pred = bag_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = bag_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier 

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1) 

rnd_clf.fit(train_x, train_y)

In [ ]:
#Train accuracy

train_y_pred = rnd_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = rnd_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

### Feature Importance

In [ ]:
rnd_clf.feature_importances_

In [ ]:
# Round to two decimals
np.round(rnd_clf.feature_importances_,2)

# Extra-Trees

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier 

ext_clf = ExtraTreesClassifier(n_estimators=500,  n_jobs=-1) 

ext_clf.fit(train_x, train_y)

In [ ]:
#Train accuracy

train_y_pred = ext_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = ext_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

# AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier 

#Create Adapative Boosting with Decision Stumps (depth=1)
ada_clf = AdaBoostClassifier( 
            DecisionTreeClassifier(max_depth=1), n_estimators=500, 
            learning_rate=0.1) 


ada_clf.fit(train_x, train_y)

In [ ]:
#Train accuracy

train_y_pred = ada_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = ada_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

# Gradient Boosting

In [ ]:
#Use GradientBoosting

from sklearn.ensemble import GradientBoostingClassifier

gbclf = GradientBoostingClassifier(max_depth=2, n_estimators=100, learning_rate=0.1) 

gbclf.fit(train_x, train_y)

In [ ]:
#Train accuracy

train_y_pred = gbclf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = gbclf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

# Stochastic Gradient Boosting

In [ ]:
#Train on 75% of the sample only

gbclf = GradientBoostingClassifier(max_depth=2, n_estimators=100, 
                                   learning_rate=0.1, subsample=0.75) 

gbclf.fit(train_x, train_y)

In [ ]:
#Train accuracy

train_y_pred = gbclf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = gbclf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

# Selecting the number of estimators

In [ ]:
for x in range(1,30):
    gbclf = GradientBoostingClassifier(max_depth=3, n_estimators=x, learning_rate=1.0) 
    gbclf.fit(train_x, train_y.ravel())
    
    train_predictions = gbclf.predict(train_x)
    test_predictions = gbclf.predict(test_x)
    
    train_accuracy = round(accuracy_score(train_y, train_predictions),4)
    test_accuracy = round(accuracy_score(test_y, test_predictions),4)
    
    print('# Estimators = {}'.format(x) + "     " + 'Train accuracy = {}'.format(train_accuracy) + "   "
         'Test accuracy = {}'.format(test_accuracy))

# Early Stopping

In [ ]:
#Notice that learning rate and tol are high to see early stopping
gbclf = GradientBoostingClassifier(max_depth=2, n_estimators=100, 
                                   learning_rate=1, 
                                  tol=0.1, n_iter_no_change=5, validation_fraction=0.2,
                                  verbose=1) 

gbclf.fit(train_x, train_y.ravel())

In [ ]:
gbclf.n_estimators_

In [ ]:
#Train accuracy

train_y_pred = gbclf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = gbclf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))

# Extreme Gradient Boosting (XGBoost)

First, start Anaconda Command Prompt, and type `pip install xgboost` to install XGBoost. Otherwise, default Anaconda installation doesn't include XGBoost. 

In [ ]:
import xgboost

In [ ]:
xgb_clf = xgboost.XGBClassifier()

xgb_clf.fit(train_x, train_y)

In [ ]:
#Train accuracy

train_y_pred = xgb_clf.predict(train_x)

train_acc = accuracy_score(train_y, train_y_pred)

print('Train acc: {}' .format(train_acc))

In [ ]:
#Test accuracy

test_y_pred = xgb_clf.predict(test_x)

test_acc = accuracy_score(test_y, test_y_pred)

print('Test acc: {}' .format(test_acc))